In [1]:
import joblib

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import cv2
import joblib
from tensorflow.keras.models import load_model

# --- Load model, scaler, and templates ---
model = load_model("best_model.keras")
scaler = joblib.load("scaler.pkl")

# Load kill emblem templates (in grayscale)
emblem_templates = [
    cv2.imread('../Resources/skull.png', 0),
    cv2.imread('../Resources/skull-1.png', 0)
]

# --- Feature Extraction ---
def crop_to_kill_emblem_region(frame):
    xmin, ymin, xmax, ymax = 770, 684, 832, 751
    roi = frame[ymin:ymax, xmin:xmax]
    return cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

def detect_kill_emblem(roi, templates):
    for template in templates:
        if roi.shape[0] < template.shape[0] or roi.shape[1] < template.shape[1]:
            roi = cv2.resize(roi, (template.shape[1], template.shape[0]))
        result = cv2.matchTemplate(roi, template, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(result)
        if max_val >= 0.6:
            return True
    return False

def extract_features(video_path, templates):
    cap = cv2.VideoCapture(video_path)
    crosshair_movement, motion_density, frame_count = 0, 0, 0
    prev_frame = None
    takedown_detected = False

    if not cap.isOpened():
        return None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame is None:
            break

        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            diff = cv2.absdiff(prev_frame, gray)
            crosshair_movement += np.sum(diff)
            motion_density += np.mean(diff > 50)

        roi = crop_to_kill_emblem_region(frame)
        if detect_kill_emblem(roi, templates):
            takedown_detected = True

        prev_frame = gray

    cap.release()

    avg_crosshair = crosshair_movement / frame_count if frame_count else 0
    avg_motion = motion_density / frame_count if frame_count else 0

    return avg_crosshair, avg_motion, int(takedown_detected)

# --- Video Analysis ---
def analyze_video(video_path):
    try:
        features = extract_features(video_path, emblem_templates)
        if features is None:
            messagebox.showerror("Error", "Could not extract features from video.")
            return

        crosshair_movement, motion_density, score_takedown = features

        # Scale only continuous features
        continuous_features = np.array([[crosshair_movement, motion_density]])
        scaled = scaler.transform(continuous_features)

        # Append the binary feature (score_takedown)
        input_vector = np.hstack([scaled, [[score_takedown]]])  # Combine the features into one array (1, 3)

        # Predict using the model
        prediction = model.predict(input_vector)[0][0]
        is_bad = prediction > 0.5

        if is_bad:
            messagebox.showinfo("Gameplay Analysis",
                                "⚠️ Bad Gameplay Detected!\n\n"
                                "🔧 Tips:\n"
                                "- Reduce crosshair movement.\n"
                                "- Improve consistency in motion.\n"
                                "- Practice tracking moving targets.")
        else:
            messagebox.showinfo("Gameplay Analysis",
                                "✅ Good Gameplay Detected!\n\n"
                                "🔥 Keep it up! Consider refining your movement for peak performance.")
    except Exception as e:
        messagebox.showerror("Analysis Error", f"Error during analysis:\n{e}")

# --- Upload Button ---
def upload_video():
    file_path = filedialog.askopenfilename(filetypes=[("MP4 Files", "*.mp4")])
    if file_path:
        analyze_video(file_path)

# --- GUI Setup ---
def main_gui():
    root = tk.Tk()
    root.title("🎮 AI Gameplay Analyzer")
    root.geometry("800x500")
    root.configure(bg="#0e0e0e")

    tk.Label(root, text="AI Gameplay Analyzer", font=("Orbitron", 32, "bold"), fg="#00ffcc", bg="#0e0e0e").pack(pady=30)

    tk.Button(root,
              text="Upload Gameplay Video",
              font=("Arial", 18, "bold"),
              bg="#1e90ff",
              fg="white",
              padx=30,
              pady=10,
              relief="ridge",
              command=upload_video).pack(pady=20)

    try:
        logo = Image.open("../Resources/Splashscreen.png")
        logo = logo.resize((300, 150), Image.Resampling.LANCZOS)
        logo_img = ImageTk.PhotoImage(logo)
        tk.Label(root, image=logo_img, bg="#0e0e0e").pack(pady=10)
        root.logo_img = logo_img
    except Exception as e:
        print(f"Logo error: {e}")

    root.mainloop()

# --- Run App ---
if __name__ == "__main__":
    main_gui()


C:\Users\PMLS\anaconda3\envs\valorant_env\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
